In [ ]:
import utils
import numpy as np
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt


pd.set_option('display.expand_frame_repr', False)

%matplotlib inline

results_path = Path("C:/Users/maler/Federico/Università/Master/Tesi/evaluation_results/")

# Scalar Utilities

In [ ]:
utility_results_path = results_path.joinpath("utility_results/")

utility_results = utils.load_results(utility_results_path)
keys = sorted(utility_results.keys(), key=utils.sorting)
utilities_table = utils.utilities_table(utility_results)
utilities_table.to_csv("tables/utilities_table.csv")

In [ ]:
utilities_table = utils.load_csv("tables/utilities_table.csv")

In [ ]:
intra_wds_table, inter_wds_table = utils.wds_tables(utility_results)
intra_wds_table.pivot(index="Model", columns="Setting", values="WD").head()

In [ ]:
inter_wds_table.pivot(index="Models", columns="Setting", values="WD").head()

In [ ]:
table = utilities_table[utilities_table["Setting"].isin(utils.STANDARD_SAMPLING_SETTINGS.intersection(utils.LINEAR_SETTINGS.union(utils.THRESHOLD_SETTINGS).union(utils.DUAL_THRESHOLD_SETTINGS)))]
table.pivot(index="Model", columns="Setting", values="Utility").head()

In [ ]:
print(utils.convert_to_latex(table.pivot(index="Model", columns="Setting", values="Utility")))

# Fixed Environment Evaluation

In [ ]:
fixed_env_results_path = results_path.joinpath("fixed_env_results/")

fixed_env_results = utils.load_results(fixed_env_results_path)
keys = sorted(fixed_env_results.keys(), key=utils.sorting)

fixed_env_unique_table = utils.uniques_table(fixed_env_results)
fixed_env_unique_table.to_csv("tables/fixed_env_unique_table.csv")
fixed_env_non_dominated_table = utils.non_dominated_table(fixed_env_results)
fixed_env_non_dominated_table.to_csv("tables/fixed_env_non_dominated_table.csv")

In [ ]:
fixed_env_unique_table = utils.load_csv("tables/fixed_env_unique_table.csv")
fixed_env_non_dominated_table = utils.load_csv("tables/fixed_env_non_dominated_table.csv")

In [ ]:
table = fixed_env_unique_table[fixed_env_unique_table["Setting"].isin(utils.STANDARD_SAMPLING_SETTINGS.intersection(utils.LINEAR_SETTINGS.union(utils.THRESHOLD_SETTINGS).union(utils.DUAL_THRESHOLD_SETTINGS)))]
table.pivot(index="Model", columns="Setting", values="Uniques").head()

In [ ]:
print(utils.convert_to_latex(table.pivot(index="Model", columns="Setting", values="Uniques")))

In [ ]:
table = fixed_env_non_dominated_table[fixed_env_non_dominated_table["Setting"].isin(utils.STANDARD_SAMPLING_SETTINGS.intersection(utils.LINEAR_SETTINGS.union(utils.THRESHOLD_SETTINGS).union(utils.DUAL_THRESHOLD_SETTINGS)))]
table.pivot(index="Model", columns="Setting", values="Non-Dominated").head()

In [ ]:
print(utils.convert_to_latex(table.pivot(index="Model", columns="Setting", values="Non-Dominated")))

# Reward Vector Evaluation

In [ ]:
reward_vector_results_path = results_path.joinpath("reduced_reward_vector/")

reward_vector_results = utils.load_results(reward_vector_results_path)
keys = sorted(reward_vector_results.keys(), key=utils.sorting)
print("Loaded")
reward_vector_unique_table = utils.uniques_table(reward_vector_results)
reward_vector_unique_table.to_csv("tables/reward_vector_unique_table.csv")
print("Unique table done")
reward_vector_non_dominated_table = utils.non_dominated_table(reward_vector_results)
reward_vector_non_dominated_table.to_csv("tables/reward_vector_non_dominated_table.csv")

In [ ]:
reward_vector_unique_table = utils.load_csv("tables/reward_vector_unique_table.csv")
reward_vector_non_dominated_table = utils.load_csv("tables/reward_vector_non_dominated_table.csv")

In [ ]:
table = reward_vector_unique_table[reward_vector_unique_table["Setting"].isin(utils.STANDARD_SAMPLING_SETTINGS)]
table.pivot(index="Model", columns="Setting", values="Uniques").head()

In [ ]:
print(utils.convert_to_latex(table.pivot(index="Model", columns="Setting", values="Uniques")))

In [ ]:
table = reward_vector_non_dominated_table[reward_vector_non_dominated_table["Setting"].isin(utils.STANDARD_SAMPLING_SETTINGS.intersection(utils.LINEAR_SETTINGS.union(utils.THRESHOLD_SETTINGS).union(utils.DUAL_THRESHOLD_SETTINGS)))]
table.pivot(index="Model", columns="Setting", values="Non-Dominated").head()

In [ ]:
print(utils.convert_to_latex(table.pivot(index="Model", columns="Setting", values="Non-Dominated")))

# Agents' training plot

In [ ]:
path = Path("C:/Users/maler/Federico/Università/Master/Tesi/experiments_results/")

ENVS = [
    "cum_linear_threshold_env", "cum_linear_dual_threshold_env", "cum_linear_env", "cum_target_env",
    "cum_dual_threshold_env", "cum_threshold_env", "linear_dual_threshold_env", "linear_threshold_env", "linear_env",
    "target_env", "dual_threshold_env", "threshold_env"
]
MODELS = ["64_64_model", "128_128_64_model", "256_128_128_64_64_model", "512_256_256_128_128_64_model"]


model = "512_256_256_128_128_64_model"
env = "dual_threshold_env"

utils.plot_training_history(path, model, env)